In [149]:
import regex as re
import pandas as pd

In [150]:
import yake
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger_ru')

[nltk_data] Downloading package punkt to /home/varsey-
[nltk_data]     fabuza/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/varsey-
[nltk_data]     fabuza/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     /home/varsey-fabuza/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_ru is already up-to-
[nltk_data]       date!


True

In [151]:
from collections import Counter

In [152]:
from pymystem3 import Mystem
ms = Mystem()

In [153]:
data = pd.read_csv('data.csv')

In [154]:
data.head()

,Unnamed: 0,client_id,prof_id,job,months,description
0,0,0,1,бригадир комплексной бригады,10000,"Выращивание с/х культур:оз.пшеница,оз.ячмень,я..."
1,1,1,11,Заместитель генерального директора по общим во...,9,Решение вопросов касающихся производственно - ...
2,2,1,10,Инженер - механик,8,Перерегистрация автотракторной техники. Списан...
3,3,1,9,Заведующий машинным двором,26,"Ремонт автомобилей, тракторов и с/х техники, с..."
4,4,1,8,Заведующий прудовым хозяйством,7,Руководство коллективом прудового хозяйства. В...


In [155]:
flt_chars = [
    ')', '(', ',', '!', ' .', ':', '|', '>', '<', '*', '[', ']', '...', './', '/\n', '\xa0',
    '«', '»', '_', '?', '~', '=', '\\', ';', '\n', '``', "''", '✅', '/n', '/', '..', '.', '`', "'", '-',
    '✔', '⇒', '"', '$', '%', '&', "'", ',', '^', '`', '{', '}', "('", '. ', ' . ',
]

In [156]:
def process_item(x: str):
    for char in flt_chars:
        x = x.replace(char, ' ').lower()
    item = ' '.join(re.split(r'(\d+)', x))
    while item.count(2 * " ") > 0:
        item = item.replace(2 * " ", " ")
    return item.strip(' ').replace(' и ', ' ')

In [157]:
stop_words = nltk.corpus.stopwords.words('russian') + nltk.corpus.stopwords.words('english')
exclude = [x for x in set(stop_words + flt_chars) if len(x) > 0]

In [158]:
def process(x):
    words = nltk.word_tokenize(x)
    words = [process_item(word) for word in words if word not in exclude]

    lemmetized = []
    for word in words:
        lemmetized.append(' '.join(ms.lemmatize(word)).strip(' \n'))
        
    return process_item(' '.join(lemmetized))

In [159]:
text = data.sample(1)['description'].values[0]
print(text)
process_item(' '.join(ms.lemmatize(text)).strip(' \n'))

Обязанности: - составление технологических карт, севооборотов; - расчет норм внесения семян, удобрений, СЗР; - организация процесса выращивания и сбора зерновых и бобовых культур (озимая пшеница, ячмень, кукуруза, рапс, горох); - организация процесса выращивания и сбора кормовых культур; - автоматизация процессов заготовки и уборки кормов, закладка сырья в силосные траншеи; - внедрение и автоматизация процедур точечного земледелия; - контроль расхода топлива, посевного материала, удобрений с помощью системы ГЛОНАСС; Достижения: - рост урожайности; - эффективная организация проведения бесперебойной предпосевной, посевной, уборочной кампаний.


'обязанность составление технологический карта севооборот расчет норма внесение семя удобрение сзр организация процесс выращивание сбор зерновой бобовый культура озимый пшеница ячмень кукуруза рапс горох организация процесс выращивание сбор кормовой культура автоматизация процесс заготовка уборка корм закладка сырье в силосный траншея внедрение автоматизация процедура точечный земледелие контроль расход топливо посевной материал удобрение с помощь система глонасс достижение рост урожайность эффективный организация проведение бесперебойный предпосевной посевной уборочный кампания'

In [160]:
process(text)

'обязанность составление технологический карта севооборот расчет норма внесение семя удобрение сзр организация процесс выращивание сбор зернов бобовый культура озимый пшеница ячмень кукуруза рапс горох организация процесс выращивание сбор кормовой культура автоматизация процесс заготовка уборка корм закладка сырье силосный траншея внедрение автоматизация процедура точечный земледелие контроль расход топливо посевной материал удобрение помощь система глонасс достижение рост урожайность эффективный организация проведение бесперебойный предпосевной посевной уборочный кампания'

In [161]:
kw_extractor = yake.KeywordExtractor(
    lan='ru',
    n=2,
    dedupLim=0.9,
    dedupFunc='seqm',
    windowsSize=2,
    top=2,
    features=None
)

for job in data.sample(20)['job']:
    keywords = kw_extractor.extract_keywords(process(job))
    print(job)
    for kw in keywords:
        print(kw)
    print()
    print([x[0] for x in keywords])
    print()
    print()

Агроном овощевод
('агроном овощевод', 0.02570861714399338)
('агроном', 0.15831692877998726)

['агроном овощевод', 'агроном']


Агроном отделения
('агроном отделение', 0.02570861714399338)
('агроном', 0.15831692877998726)

['агроном отделение', 'агроном']


Управляющая фермой до мая 2016, зоотехник по кормам с мая 2016г.
('управлять ферма', 0.02391462167226668)
('зоотехник корм', 0.039481144315219374)

['управлять ферма', 'зоотехник корм']


Главный агроном
('главный агроном', 0.02570861714399338)
('главный', 0.15831692877998726)

['главный агроном', 'главный']


ведущий специалист кузовного производства.
('ведущий специалист', 0.04940384002065631)
('кузовной производство', 0.04940384002065631)

['ведущий специалист', 'кузовной производство']


Ветеринарный врач
('ветеринарный врач', 0.02570861714399338)
('ветеринарный', 0.15831692877998726)

['ветеринарный врач', 'ветеринарный']


Технический директор
('технический директор', 0.02570861714399338)
('технический', 0.15831692877998726)

[

In [162]:
def extract(x):
    keywords = kw_extractor.extract_keywords(process(x))
    if len(keywords) > 0:
        return [str(x[0]) for x in keywords]
    elif len(keywords) == 0:
        return ''

In [163]:
data["jobs_selected"] = data["job"].apply(extract)

In [164]:
data.sample(25)

,Unnamed: 0,client_id,prof_id,job,months,description,jobs_selected
884,884,179,4,штабелировщик металла,18,прием готовой продукции на склад и отгрузка в ...,"[штабелировщик металл, штабелировщик]"
1532,1532,335,8,Заведующий Госсортучастка,10000,Руководство отделом по сортоиспытанию с/х куль...,"[заведующий госсортучасток, заведующий]"
1779,1779,389,2,Химик-хроматографист,8,"Работа с аппаратно-программный комплекс""Хромат...","[химик хроматографист, химик]"
3220,3220,737,8,Агроном,10000,"планирование, организация, контроль выполнения...",[агроном]
557,557,121,7,Ведущий ветеринарный врач,10000,Оформление ветеринарных сопроводительных докум...,"[ведущий ветеринарный, ветеринарный врач]"
4156,4156,968,4,Водитель автобуса 1 класса,13,Перевозка пассажиров,"[водитель автобус, водитель]"
3790,3790,875,4,Сервисный инженер,37,Выполнение гарантийного и пост гарантийного об...,"[сервисный инженер, сервисный]"
1216,1216,262,5,Начальник ветеринарной службы,18,Руководство ветеринарной службой. Штат 50 чело...,"[начальник ветеринарный, ветеринарный служба]"
2420,2420,550,5,"Водитель категории В, С",13,Август 2010 года переведен на должность машини...,"[водитель категория, водитель]"
4048,4048,934,3,Главный инжеенер,31,Обеспечение бесперебойной работы всех систем к...,"[главный инжеенер, главный]"


In [165]:
data = data.explode(['jobs_selected'])

In [166]:
data.shape

(7850, 7)

In [167]:
data["jobs_selected"] = data["jobs_selected"].astype('str')

In [168]:
data.sample(25)

,Unnamed: 0,client_id,prof_id,job,months,description,jobs_selected
1797,1797,393,1,Механик отдела технического контроля,151,Выпуск на линию технически исправного транспор...,механик отдел
357,357,81,3,ведущий агроном-эксперт,79,"Сертификация продукции растениеводства, монито...",ведущий агроном
1502,1502,328,11,Директор Донецкой областной государственной ла...,37,Должностные обязанности: организация и проведе...,ветеринарный медицина
781,781,160,5,Водитель 3 класса,12,"Выполнял обязанности водителя, обслуживание не...",класс
1894,1894,417,9,Методист,10000,Разработка методических программ по проведению...,методист
3189,3189,732,11,Главный механик,8,Руководитель подразделения предприятия по эксп...,главный механик
4298,4298,994,3,Заведующий кирпичным заводом.,10000,Организация труда по производству глиняного ки...,заведующий кирпичный
1491,1491,326,3,Командир отделения взвода артиллерийской разве...,13,обязанности заместителя командира взвода и ста...,командир отделение
1174,1174,250,1,Механик,10000,Ремонт оборудования на производстве,механик
3755,3755,870,2,Заместитель главного ветеринарного врача комби...,51,Мастер консервного цеха и цеха детского питани...,врач комбинат


In [169]:
Counter(" ".join(data['job']).split()).most_common(100)

[('по', 1003),
 ('врач', 772),
 ('Главный', 754),
 ('Ветеринарный', 518),
 ('инженер', 353),
 ('и', 352),
 ('ветеринарный', 348),
 ('Начальник', 346),
 ('агроном', 344),
 ('отдела', 280),
 ('Инженер', 236),
 ('Агроном', 222),
 ('Старший', 192),
 ('участка', 190),
 ('Менеджер', 182),
 ('механик', 180),
 ('директора', 170),
 ('Машинист', 168),
 ('Механик', 167),
 ('Заместитель', 154),
 ('Мастер', 152),
 ('директор', 141),
 ('разряда', 134),
 ('специалист', 134),
 ('Заведующий', 133),
 ('Специалист', 133),
 ('главный', 132),
 ('Водитель', 128),
 ('Ведущий', 126),
 ('Инженер-механик', 124),
 ('Руководитель', 124),
 ('зоотехник', 122),
 ('Тракторист-машинист', 116),
 ('цеха', 110),
 ('Оператор', 103),
 ('службы', 102),
 ('ремонту', 94),
 ('-', 89),
 ('оборудования', 88),
 ('начальник', 86),
 ('производства', 86),
 ('с', 85),
 ('менеджер', 84),
 ('продажам', 84),
 ('Директор', 79),
 ('Сервисный', 78),
 ('Управляющий', 76),
 ('Помощник', 72),
 ('Технолог', 70),
 ('Слесарь', 69),
 ('в', 68),
 

In [170]:
conter_dict = Counter(",".join(data['jobs_selected']).split(',')).most_common(1000)

In [171]:
data["counted_jobs_selected"] = data["jobs_selected"]

In [172]:
dct_corr = {k: v for k, v in dict(conter_dict).items() if isinstance(v, int)}

In [173]:
dct_corr

{'ветеринарный врач': 399,
 'главный': 270,
 'ветеринарный': 226,
 'агроном': 198,
 'инженер': 174,
 'главный агроном': 115,
 'водитель': 115,
 'механик': 109,
 'инженер механик': 89,
 'главный инженер': 87,
 'главный ветеринарный': 86,
 'тракторист': 85,
 'начальник': 79,
 'зоотехник': 67,
 'тракторист машинист': 64,
 'мастер': 64,
 'машинист': 59,
 'директор': 57,
 'менеджер': 56,
 'главный зоотехник': 50,
 'слесарь': 43,
 'сервисный инженер': 42,
 'заместитель директор': 41,
 'продавец': 39,
 'менеджер продажа': 39,
 'сервисный': 37,
 'механизатор': 36,
 'старший': 36,
 'машинист экскаватор': 35,
 'технолог': 35,
 'заведующий': 34,
 'главный механик': 34,
 'управлять': 33,
 'начальник участок': 33,
 'генеральный директор': 31,
 'начальник отдел': 31,
 'руководитель': 29,
 'водитель экспедитор': 29,
 'старший ветеринарный': 28,
 'продавец консультант': 28,
 'директор производство': 28,
 'защита растение': 26,
 'ведущий': 26,
 'инженер технолог': 25,
 'ведущий специалист': 24,
 'научн

In [174]:
data = data.replace({"counted_jobs_selected": dct_corr})

In [175]:
data.shape

(7850, 8)

In [176]:
data.dropna(inplace=True)

In [177]:
# data = data[data['counted_jobs_selected'].isin(dct_corr.keys())]
data.shape

(7848, 8)

In [178]:
data['counted_jobs_selected'] = pd.to_numeric(data['counted_jobs_selected'], errors='coerce')

In [179]:
data.dropna(subset = ['counted_jobs_selected'])
data.shape

(7848, 8)

In [180]:
data = data[data.counted_jobs_selected > 20]

In [181]:
data = data.drop_duplicates(subset=['client_id', 'prof_id', 'jobs_selected'])

In [182]:
data.sample(25)

,Unnamed: 0,client_id,prof_id,job,months,description,jobs_selected,counted_jobs_selected
27,27,10,8,Ветеринарный врач,10000,"Разнообразная лечебная деятельность (собаки, к...",ветеринарный врач,399.0
1289,1289,282,4,Сервисный инженер,25,"Тех поддержка, обслуживание оборудования, взаи...",сервисный инженер,42.0
562,562,121,2,Главный ветеринарный врач Рай СББЖ,116,"Осуществлению контроля за производством , хран...",главный ветеринарный,86.0
3371,3371,777,4,Торговый представитель,6,Продажи товаров по торговым точкам. Кантроль д...,торговый представитель,23.0
1628,1628,356,5,Мастер-агроном,3,Контроль над бригадой рабочих паркового хозяйс...,мастер,64.0
3839,3839,886,7,Инженер-механик,93,- составление графиков обслуживания техники - ...,инженер,174.0
4053,4053,936,1,ветеринарный врач,172,"профилактика, диагностика, лечение сельскохозя...",ветеринарный врач,399.0
3962,3962,916,3,Торговый представитель,14,"Поиск,формирование заказа,закупка,контроль за ...",торговый представитель,23.0
292,292,67,5,Заместитель директора по административно-хозяй...,14,Руководство ремонтно-техническим и обслуживающ...,заместитель директор,41.0
2644,2644,601,1,"главный технолог, зам директора",138,"Удовлетворять требования потребителей, обеспеч...",главный технолог,21.0


In [183]:
len(data.client_id.unique())

897

In [184]:
data[data.client_id == 815]

,Unnamed: 0,client_id,prof_id,job,months,description,jobs_selected,counted_jobs_selected
3524,3524,815,6,Продавец,17,"Выкладка товара по планограмме, консультирован...",продавец,39.0
3525,3525,815,5,Сервисный инженер,18,"Ремонт, установка, тех. обслуживание Широкофор...",сервисный инженер,42.0
3525,3525,815,5,Сервисный инженер,18,"Ремонт, установка, тех. обслуживание Широкофор...",сервисный,37.0


In [185]:
dict(nltk.pos_tag(process(text).split(), lang='rus'))

{'обязанность': 'V',
 'составление': 'S',
 'технологический': 'A=m',
 'карта': 'S',
 'севооборот': 'S',
 'расчет': 'V',
 'норма': 'S',
 'внесение': 'S',
 'семя': 'S',
 'удобрение': 'S',
 'сзр': 'S',
 'организация': 'S',
 'процесс': 'S',
 'выращивание': 'S',
 'сбор': 'S',
 'зернов': 'S',
 'бобовый': 'A=m',
 'культура': 'S',
 'озимый': 'V',
 'пшеница': 'S',
 'ячмень': 'S',
 'кукуруза': 'S',
 'рапс': 'S',
 'горох': 'S',
 'кормовой': 'A=f',
 'автоматизация': 'S',
 'заготовка': 'S',
 'уборка': 'S',
 'корм': 'S',
 'закладка': 'S',
 'сырье': 'S',
 'силосный': 'A=m',
 'траншея': 'S',
 'внедрение': 'S',
 'процедура': 'S',
 'точечный': 'A=m',
 'земледелие': 'S',
 'контроль': 'S',
 'расход': 'S',
 'топливо': 'ADV',
 'посевной': 'S',
 'материал': 'S',
 'помощь': 'S',
 'система': 'S',
 'глонасс': 'S',
 'достижение': 'S',
 'рост': 'S',
 'урожайность': 'S',
 'эффективный': 'A=m',
 'проведение': 'S',
 'бесперебойный': 'A=m',
 'предпосевной': 'A=f',
 'уборочный': 'V',
 'кампания': 'S'}

In [186]:
data["is_adj"] = data["jobs_selected"]

In [187]:
def check_adj(x):
    if len(x.split()) > 1:
        return x
    else:
        spec = nltk.pos_tag([x], lang='rus')
        print(spec, len(x.split()))
        if len(spec) == 1:
            if spec[0][1] == 'A=m':
                return spec[0][1]
            else:
                return spec[0][0]
        else:
            return x

In [188]:
data["is_adj"] = data["is_adj"].apply(lambda x : check_adj(x))

[('инженер', 'S')] 1
[('механик', 'S')] 1
[('агроном', 'A=n')] 1
[('главный', 'A=m')] 1
[('инженер', 'S')] 1
[('машинист', 'S')] 1
[('машинист', 'S')] 1
[('машинист', 'S')] 1
[('зоотехник', 'S')] 1
[('заведующий', 'V')] 1
[('ветеринарный', 'A=m')] 1
[('тракторист', 'S')] 1
[('ветеринарный', 'A=m')] 1
[('начальник', 'S')] 1
[('ветеринарный', 'A=m')] 1
[('тракторист', 'S')] 1
[('ветеринарный', 'A=m')] 1
[('ветеринарный', 'A=m')] 1
[('главный', 'A=m')] 1
[('управлять', 'V')] 1
[('продавец', 'S')] 1
[('водитель', 'S')] 1
[('менеджер', 'S')] 1
[('машинист', 'S')] 1
[('заведующий', 'V')] 1
[('ветеринарный', 'A=m')] 1
[('ветеринарный', 'A=m')] 1
[('тракторист', 'S')] 1
[('руководитель', 'S')] 1
[('продавец', 'S')] 1
[('главный', 'A=m')] 1
[('директор', 'S')] 1
[('директор', 'S')] 1
[('директор', 'S')] 1
[('главный', 'A=m')] 1
[('технолог', 'S')] 1
[('инженер', 'S')] 1
[('заведующий', 'V')] 1
[('продавец', 'S')] 1
[('зоотехник', 'S')] 1
[('продавец', 'S')] 1
[('сервисный', 'A=m')] 1
[('инженер

In [189]:
data[data.client_id == 815]

,Unnamed: 0,client_id,prof_id,job,months,description,jobs_selected,counted_jobs_selected,is_adj
3524,3524,815,6,Продавец,17,"Выкладка товара по планограмме, консультирован...",продавец,39.0,продавец
3525,3525,815,5,Сервисный инженер,18,"Ремонт, установка, тех. обслуживание Широкофор...",сервисный инженер,42.0,сервисный инженер
3525,3525,815,5,Сервисный инженер,18,"Ремонт, установка, тех. обслуживание Широкофор...",сервисный,37.0,A=m


In [190]:
check_adj('сервисный')

[('сервисный', 'A=m')] 1


'A=m'

In [191]:
data = data[data.is_adj != 'A=m']

In [212]:
data

,Unnamed: 0,client_id,prof_id,job,months,description,jobs_selected,counted_jobs_selected,is_adj
1,1,1,11,Заместитель генерального директора по общим во...,9,Решение вопросов касающихся производственно - ...,заместитель генеральный,21.0,заместитель генеральный
2,2,1,10,Инженер - механик,8,Перерегистрация автотракторной техники. Списан...,инженер механик,89.0,инженер механик
2,2,1,10,Инженер - механик,8,Перерегистрация автотракторной техники. Списан...,инженер,174.0,инженер
11,11,1,1,Механик кормоцеха,3,"Ремонт, обслуживание оборудования и тракторов ...",механик,109.0,механик
12,12,2,1,агроном семеновод,10000,обязанности смотреть за качеством семенного ма...,агроном,198.0,агроном
...,...,...,...,...,...,...,...,...,...
4325,4325,999,9,специалист видеонаблюдения,4,- осуществление видеонаблюдения - контроль за ...,специалист,22.0,специалист
4327,4327,999,7,Начальник участка водоснабжения,21,"-обеспечение бесперебойной работы транспорта, ...",начальник участок,33.0,начальник участок
4331,4331,999,3,Менеджер по продажам услуг,9,"- ведение документооборота, проверка отчетов у...",менеджер продажа,39.0,менеджер продажа
4332,4332,999,2,Главный механик,73,"- организация, управление автопарком, обеспече...",главный механик,34.0,главный механик


In [254]:
data[['client_id', 'prof_id', 'jobs_selected', 'counted_jobs_selected', 'job', 'counted_jobs_selected', 'description', 'months']].to_csv('data_selected.csv')